In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import concat
from pyspark.sql.functions import regexp_replace

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1651992266925_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark = SparkSession\
    .builder \
    .appName("Kmeans Clustering Lab") \
    .getOrCreate()

We are reading data from the CSV file and putting it into a Spark dataframe and letting it infer the schema from the header.

In [ ]:
df = spark.read.csv('s3://cs496-proga1-storage/covid_data/metadata.csv',
                    sep=',',
                    inferSchema=True,
                    header=True)

In [ ]:
df.printSchema()

We only want to keep the tile and abstract columns. We drop the rest from the dataframe.

In [ ]:
df = df.drop('cord_uid', 'sha', 'source_x', 'doi', 'pmcid', 'pubmed_id', 'license', 'publish_time', 'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files', 'url', 's2_id')
# Display the dataframe.
df.show()

In [ ]:
# We drop the null values from the dataframe.
df = df.na.drop()
df.show()

In [ ]:
# We concatenate the columns into one.
dframe = df.select(concat(df.title,df.abstract))
dframe = dframe.withColumnRenamed("concat(title, abstract)","data")
dframe.show(truncate=False)

In [ ]:
# What is the schema of the new datafrome?
dframe.printSchema()

In [ ]:
# We strip all the punctuation and whitespaces in the data column.
dframe = dframe.withColumn("data", regexp_replace(col("data"), "[^a-zA-Z]+", ""))
dframe.show(truncate=False)

In [ ]:
# Turn the string column into an array column.
from pyspark.sql.functions import split
dframe2 = dframe.select(split(col("data"),",").alias("d_array")) \
    .drop("data")
dframe2.printSchema()
dframe2.show(truncate=False)

In [ ]:
# Then we need to use a UDF to convert the string array into a vectors.
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vect_udf = udf(lambda 1: Vectors.dense(1), VectorUDT())
dframe_vectors = dframe2.select(dframe2["d_array"], list_to_vect_udf(dframe2["d_array"]).alias["data_vec"])

In [ ]:
# Here we try to use Vector Assembler for the columns.
from pyspark.ml.feature import VectorAssembler
cols = dframe_vectors.columns
assemble = VectorAssembler(inputCols=cols, outputCol='features')
assembled_data = assemble.transform(dframe_vectors)
assembled_data.show(truncate=False)